In [165]:
from pathlib import Path
import numpy as np
import sys

sys.setrecursionlimit(100_000)

Point = tuple[int,int]

UP = 1
RIGHT = 2
LEFT = 4
DOWN = 8

transition = (
    (+0, +0),  # 0
    ( 0, -1),  # 1
    (+1, +0),  # 2
    (+0, +0),  # 3
    (-1, +0),  # 4
    (+0, +0),  # 5
    (+0, +0),  # 6
    (+0, +0),  # 7
    (+0, +1),  # 8
)

def get_opposite_direction(d: int) -> int:
    return 8//d

def present_nicely(maze: str):
    nicely_presented = maze.replace('F', '┌').replace('J', '┘').replace('L', '└').replace('7', '┐').replace('-','─').replace('|', '│').replace('.', ' ').replace('S', 'X')
    print(nicely_presented)

from_file = Path('input.txt').read_text()
#from_file = '''\
# ..........
# .S------7.
# .|F----7|.
# .||....||.
# .||....||.
# .|L-7F-J|.
# .|..||..|.
# .L--JL--J.
# ..........'''
# from_file = '''\
# .F----7F7F7F7F-7....
# .|F--7||||||||FJ....
# .||.FJ||||||||L7....
# FJL7L7LJLJ||LJ.L-7..
# L--J.L7...LJS7F-7L7.
# ....F-J..F7FJ|L7L7L7
# ....L7.F7||L7|.L7L7|
# .....|FJLJ|FJ|F7|.LJ
# ....FJL-7.||.||||...
# ....L---J.LJ.LJLJ...'''
#present_nicely(from_file)

In [166]:
def map_string_to_list(s: str) -> tuple[list[list[int]], Point]:
    '''Given all input file as string, return it as a 2D map and starting point
    '''
    l = []
    starting_point = (-1,-1)
    for y, line in enumerate(s.splitlines()):
        z = [
            {
                "F": RIGHT | DOWN,
                "J": UP | LEFT,
                "L": UP | RIGHT,
                "7": LEFT | DOWN,
                "-": LEFT | RIGHT,
                "|": UP | DOWN,
                "S": 0,
                ".": 0
            }[x]
            for x in line.strip()
        ]
        starting_point_x = line.find('S')
        if starting_point_x >= 0:
            starting_point = (starting_point_x, y)
            z[starting_point_x] = UP|DOWN  # Look at the input and determine the starting point value
        l.append(z)
    return (l, starting_point)

maze, starting_point = map_string_to_list(from_file)

In [167]:
Y = len(maze); X = len(maze[0])
(x, y) = starting_point
clean_maze = np.zeros((Y, X), dtype=np.uint8)  # for part 2
direction = DOWN  # looking at both the example and input.txt, DOWN is available
steps = 0
while True:
    dx, dy = transition[direction]
    x += dx
    y += dy
    steps += 1
    clean_maze[y,x] = maze[y][x]
    if (x, y) == starting_point:
        break
    direction = maze[y][x] & ~get_opposite_direction(direction)

furthest_point = (steps + 1) // 2
furthest_point

6870

# Part 2

In [169]:
def clean_maze_to_3x3_maze(maze: np.ndarray) -> np.ndarray:
    '''Generate an expanded maze where each cell becomes 3x3
    '''
    Y, X = maze.shape
    m = np.zeros((Y*3, X*3), dtype=np.uint8)
    for y in range(Y):
        for x in range(X):
            if maze[y,x] == 0:
                continue
            y3, x3 = y*3+1, x*3+1
            m[y3, x3] = 1
            if maze[y,x] & UP:
                m[y3-1, x3] = 1
            if maze[y,x] & DOWN:
                m[y3+1, x3] = 1
            if maze[y,x] & LEFT:
                m[y3, x3-1] = 1
            if maze[y,x] & RIGHT:
                m[y3, x3+1] = 1
    return m

    

def flood_fill(x: int, y: int, maze: np.ndarray) -> None:
    '''Given a starting point inside a 3x3 expanded maze,
    flood fill it.
    '''
    assert maze[y,x] == 0, f'{x}, {y}, {maze[y,x]}'
    Y, X = maze.shape
    maze[y,x] = 100
    if x > 0 and maze[y,x-1] == 0:
        flood_fill(x-1, y, maze)
    if x < X-1 and maze[y,x+1] == 0:
        flood_fill(x+1, y, maze)
    if y > 0 and maze[y-1,x] == 0:
        flood_fill(x, y-1, maze)
    if y < Y-1 and maze[y+1,x] == 0:
        flood_fill(x, y+1, maze)

def count_volume(clean_maze: np.ndarray, expanded: np.ndarray) -> tuple[np.ndarray, int]:
    Y, X = clean_maze.shape
    out_maze = clean_maze.copy()
    volume = 0
    for y in range(Y):
        for x in range(X):
            if expanded[y*3+1, x*3+1] == 100:
                out_maze[y, x] = 100
            if expanded[y*3+1, x*3+1] == 0:
                volume += 1
    return out_maze, volume


def print_pipe_loop(maze: np.ndarray):
    with open('kishkish.txt', 'w') as f:
        for y in range(maze.shape[0]):
            print(''.join([{0:'@', 100:'.', UP|DOWN:'│',LEFT|RIGHT:'─', UP|RIGHT:'└', RIGHT|DOWN:'┌', DOWN|LEFT:'┐', LEFT|UP:'┘'}[z] for z in maze[y, :]]), file=f)

def print_3x3_maze(maze: np.ndarray):
    for y in range(maze.shape[0]):
        print(''.join(['.' if z==100 else '*' if z==1 else ' ' for z in maze[y,:]]))

expanded_maze = clean_maze_to_3x3_maze(clean_maze)
x0, y0 = 0,0
flood_fill(x0*3+1, y0*3+1, expanded_maze)
filled_maze, volume = count_volume(clean_maze, expanded_maze)
print(volume)
print_pipe_loop(filled_maze)


287


In [117]:
present_nicely('''\
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....''')

present_nicely('''\
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........''')


           
 X───────┐ 
 │┌─────┐│ 
 ││OOOOO││ 
 ││OOOOO││ 
 │└─┐O┌─┘│ 
 │II│O│II│ 
 └──┘O└──┘ 
     O     
          
 X──────┐ 
 │┌────┐│ 
 ││OOOO││ 
 ││OOOO││ 
 │└─┐┌─┘│ 
 │II││II│ 
 └──┘└──┘ 
          
